In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from functools import reduce
import string
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dmitr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dmitr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
morph_uk = MorphAnalyzer(lang='uk')


def clear_punctuation(text):
    text = text.translate(str.maketrans('', '', string.punctuation + '«»—–-’'))
    return text.replace('\n', ' ')


def get_stopwords_remover(stop_words):
    return lambda words: [word for word in words if not word.lower() in stop_words]


def preprocessing_pipeline(steps):
    return lambda raw_text: reduce(lambda data, func: func(data), steps, raw_text)


def prepare_words(text):
    words = word_tokenize(text, language='russian')

    return [morph_uk.parse(word)[0].normal_form for word in words]

def clear_nbsp(text):
    return text.replace(' ', ' ')

In [3]:
ukr_text = pd.read_csv('data/ukr_text.csv')
ukr_text

,Id,Title,Body
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...
...,...,...,...
1117,http://k.img.com.ua/rss/ua/3194862,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...
1118,http://k.img.com.ua/rss/ua/3194633,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...
1119,http://k.img.com.ua/rss/ua/3194587,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...
1120,http://k.img.com.ua/rss/ua/3194570,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т..."


In [4]:
with open('data/stopwords_ua.txt', encoding='utf-8') as file:
    stopwords_ua = file.read().splitlines()

In [5]:
stop_words = set(stopwords_ua)
remove_stopwords = get_stopwords_remover(stop_words)
stop_words

{'кожнісінькою',
 'тудою',
 'цієї',
 'одну',
 'подеяка',
 'неначебто',
 'котримсь',
 'деінде',
 'ґ',
 'тал',
 'нічийними',
 'сик',
 'жодною',
 'такій',
 'своє',
 'напроти',
 'дещо',
 'своїх',
 'шаг',
 'бош',
 'як',
 'деколи',
 'котрихсь',
 'нічийний',
 'поблизу',
 'еч',
 'заради',
 "п'ятнадцятий",
 'анічогісінько',
 'ніякісінькій',
 'наокіл',
 'деякому',
 'отакого',
 'того',
 'аніякій',
 'сімнадцять',
 'якімсь',
 'хо',
 'скрізь',
 'мжа',
 'мало',
 'тамтієї',
 'жуз',
 'еж',
 'лат',
 'які',
 'котроїсь',
 'отака',
 'фур',
 'чиєюсь',
 'тсс',
 'більше',
 'гир',
 'році',
 'кожнісінькому',
 'моєю',
 'інакший',
 'доти',
 'на знак',
 'на підставі',
 'якимсь',
 'ніби-то',
 'там',
 'ато',
 'абичию',
 'аж',
 'іноді',
 'цоп',
 'с',
 'жоднісінькою',
 'чию',
 'оскільки',
 'оцій',
 'да',
 'всяка',
 'в силу',
 'інакшими',
 'такою',
 'бер',
 "в ім'я",
 'із-під',
 'кожних',
 'нашими',
 'трохи',
 'середи',
 'йно',
 'уздовж',
 'твоїй',
 'інші',
 'оцю',
 'кудись',
 'насеред',
 'ніхто',
 'аніякісенького',
 '

In [6]:
pipe = preprocessing_pipeline([
    clear_nbsp,
    clear_punctuation,
    prepare_words,
    remove_stopwords
])

In [7]:
ukr_text['words'] = ukr_text.Body.apply(pipe)
ukr_text

,Id,Title,Body,words
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...,"[перспективний, відкритий, ринок, бізнес, інве..."
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...,"[зявитися, відео, конфлікт, мотоцикліст, водій..."
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...,"[бессарабський, площа, вранці, четвер, 16, сер..."
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...,"[вночі, 16, серпень, пережити, найсильніший, г..."
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...,"[уночі, вкотре, накрити, негода, найсильніший,..."
...,...,...,...,...
1117,http://k.img.com.ua/rss/ua/3194862,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...,"[київський, офіс, класичний, ювелірний, дома, ..."
1118,http://k.img.com.ua/rss/ua/3194633,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...,"[20, том, орден, заслуга, називатися, почесний..."
1119,http://k.img.com.ua/rss/ua/3194587,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...,"[10, жовтень, політичний, важковаговик, табір,..."
1120,http://k.img.com.ua/rss/ua/3194570,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т...","[ізраїль, країна, займати, місце, світ, надій,..."


In [8]:
tone_dict = pd.read_csv('https://raw.githubusercontent.com/lang-uk/tone-dict-uk/master/tone-dict-uk.tsv', delimiter='\t', names=['word', 'tone'], index_col=0)
tone_dict

,tone
word,
Всевишній,1
Господь,1
Христовий,1
аборт,-1
абсурд,-1
...,...
янгол,1
яскравий,1
ясність,1


In [9]:
def calculate_sentiment(words):
    tone_words = [word for word in words if word in tone_dict.index]
    if len(set(tone_words)) < 5:
        return 0
    return sum([tone_dict.loc[word].tone if word in tone_dict.index else 0 for word in words]) / len(words)

In [10]:
ukr_text['tone'] = ukr_text.words.apply(calculate_sentiment)
ukr_text

,Id,Title,Body,words,tone
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...,"[перспективний, відкритий, ринок, бізнес, інве...",0.026163
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...,"[зявитися, відео, конфлікт, мотоцикліст, водій...",-0.090909
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...,"[бессарабський, площа, вранці, четвер, 16, сер...",-0.137255
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...,"[вночі, 16, серпень, пережити, найсильніший, г...",0.000000
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...,"[уночі, вкотре, накрити, негода, найсильніший,...",0.000000
...,...,...,...,...,...
1117,http://k.img.com.ua/rss/ua/3194862,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...,"[київський, офіс, класичний, ювелірний, дома, ...",0.042616
1118,http://k.img.com.ua/rss/ua/3194633,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...,"[20, том, орден, заслуга, називатися, почесний...",0.124682
1119,http://k.img.com.ua/rss/ua/3194587,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...,"[10, жовтень, політичний, важковаговик, табір,...",0.022042
1120,http://k.img.com.ua/rss/ua/3194570,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т...","[ізраїль, країна, займати, місце, світ, надій,...",0.018657


In [11]:
print(ukr_text.sort_values(by='tone', ascending=True).iloc[0].Body)

У Державному департаменті США заявили, що США разом з усім світом згадують жертв Голодомору і вкотре підтвердили прихильність демократії, процвітанню і суверенітету України. Про це йдеться в заяві Держдепу, передає Голос Америки. Прес-секретар Державного департаменту США Морган Ортагюс заявила: "Ми об'єднуємося з усім світом, щоб згадати невинних жертв Голодомору і підтвердити нашу прихильність демократії, процвітанню і суверенітету України". У Держдепі заявили, що Голодомор - одна з найжорстокіших трагедій 20 століття. "Шляхом навмисного захоплення української землі, врожаю і примусової колективізації, Радянський Союз призвів до масштабного голоду, смертей і приніс надзвичайні людські страждання ... Хоча ця жахлива трагедія була однією з найжорстокіших в 20 столітті, Радянському Союзу не вдалося зломити дух українського народу". Раніше повідомлялося, що в Україні відзначають День пам'яті жертв Голодоморів. По всій території України приспущено державні прапори й обмежено проведення зах

In [12]:
print(ukr_text.sort_values(by='tone', ascending=False).iloc[0].Body)

Тільки на мовних курсах у Великій Британії учень з головою може зануритися в природне мовне середовище і отримати безцінний багаж знань. Однак, чим англійська освіта на мовних курсах відрізняється від навчання в інших країнах? Що робить вивчення англійської в Великобританії настільки особливим, що кожен іноземний студент, мріє відправитися на вивчення англійської саме в цю країну? У цій статті, підготувати яку нам допомогло освітнє агентство PFI, ми пропонуємо Вам 5 цікавих фактів про мовні курси в Англії.   Концентрація на розмовних навичках Головною відмінністю мовних курсів в Англії є фокус на розвитку розмовних і комунікативних навичок абітурієнта. Кваліфіковані куратори груп планують навчальний процес таким чином, щоб теоретична частина займала 20%, а практика - 80%. Подібний підхід до навчання вперше застосували навчальні заклади Великобританії, раніше з'ясувавши, що активне обговорення і постійна мовна практика підвищують загальну успішність студента і результативність всього ку

In [13]:
ukr_text['clean_text'] = ukr_text.words.str.join(' ')
ukr_text

,Id,Title,Body,words,tone,clean_text
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...,"[перспективний, відкритий, ринок, бізнес, інве...",0.026163,перспективний відкритий ринок бізнес інвестиці...
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...,"[зявитися, відео, конфлікт, мотоцикліст, водій...",-0.090909,зявитися відео конфлікт мотоцикліст водій авто...
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...,"[бессарабський, площа, вранці, четвер, 16, сер...",-0.137255,бессарабський площа вранці четвер 16 серпень в...
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...,"[вночі, 16, серпень, пережити, найсильніший, г...",0.000000,вночі 16 серпень пережити найсильніший гроза з...
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...,"[уночі, вкотре, накрити, негода, найсильніший,...",0.000000,уночі вкотре накрити негода найсильніший дощ п...
...,...,...,...,...,...,...
1117,http://k.img.com.ua/rss/ua/3194862,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...,"[київський, офіс, класичний, ювелірний, дома, ...",0.042616,київський офіс класичний ювелірний дома лоборт...
1118,http://k.img.com.ua/rss/ua/3194633,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...,"[20, том, орден, заслуга, називатися, почесний...",0.124682,20 том орден заслуга називатися почесний знак ...
1119,http://k.img.com.ua/rss/ua/3194587,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...,"[10, жовтень, політичний, важковаговик, табір,...",0.022042,10 жовтень політичний важковаговик табір опози...
1120,http://k.img.com.ua/rss/ua/3194570,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т...","[ізраїль, країна, займати, місце, світ, надій,...",0.018657,ізраїль країна займати місце світ надій молоко...


In [14]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(ukr_text.clean_text)

In [15]:
svd_vectorizer = TruncatedSVD(n_components=100, random_state=42)
X_lsa = svd_vectorizer.fit_transform(X)

In [16]:
X_lsa

array([[ 0.22554268, -0.01841248,  0.07070091, ..., -0.02838924,
         0.01491171,  0.00846253],
       [ 0.09406398, -0.02790445, -0.0320689 , ...,  0.07981946,
         0.01932493, -0.04900203],
       [ 0.09417164, -0.02188734, -0.03898791, ...,  0.04596805,
         0.07591701, -0.02015498],
       ...,
       [ 0.20440434, -0.0483453 , -0.00387152, ..., -0.02651493,
        -0.01794904, -0.06926745],
       [ 0.21970905, -0.02814167,  0.03929289, ..., -0.01025367,
        -0.03825015, -0.04493127],
       [ 0.24004565, -0.07530466, -0.05597721, ..., -0.04029973,
         0.03659069, -0.1107553 ]])

In [17]:
def get_topic_words(vectorizer, svd, n_top_words):
    words = vectorizer.get_feature_names_out()
    topics = []
    for component in svd.components_:
        top_words_idx = np.argsort(component)[::-1][:n_top_words]
        top_words = [words[i] for i in top_words_idx]
        topics.append(top_words)
    return topics

In [18]:
topics = get_topic_words(tfidf_vectorizer, svd_vectorizer, 10)
for i, topic in enumerate(topics):
    print(f'Topic {i}: {", ".join(topic)}')

Topic 0: україна, рок, долар, компанія, росія, новий, йога, країна, область, грудень
Topic 1: нафта, біржа, долар, індекс, барель, пункт, марка, торг, ньюйоркський, brent
Topic 2: область, регіон, інвестиція, газа, млрд, грн, населення, душити, транзит, україна
Topic 3: долар, газа, транзит, росія, україна, нафтогаз, газпром, нафта, російський, барель
Topic 4: індекс, пункт, jones, dow, nasdaq, фондовий, 500, виборчий, ринок, цвк
Topic 5: виборчий, округа, цвк, комісія, депутат, народний, серпень, обраний, зареєструвати, голос
Topic 6: гонка, область, кубок, україна, регіон, збірний, підручний, естафета, змішаний, інвестиція
Topic 7: матч, динамо, штрафний, мяч, шахтар, пробити, ворота, очко, команда, збірний
Topic 8: мерседес, феррарі, феттля, хемілтон, гранпрі, боттас, себастьян, булла, льюїс, хаас
Topic 9: шоу, учасник, випуск, мастершеф, сезон, санкція, північний, росія, конкурс, страва
Topic 10: шоу, випуск, учасник, мастершеф, сезон, млрд, страва, газа, долар, кулінар
Topic 11: г

In [19]:
ukr_text['topic'] = X_lsa.argmax(axis=1)
ukr_text

,Id,Title,Body,words,tone,clean_text,topic
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...,"[перспективний, відкритий, ринок, бізнес, інве...",0.026163,перспективний відкритий ринок бізнес інвестиці...,0
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...,"[зявитися, відео, конфлікт, мотоцикліст, водій...",-0.090909,зявитися відео конфлікт мотоцикліст водій авто...,0
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...,"[бессарабський, площа, вранці, четвер, 16, сер...",-0.137255,бессарабський площа вранці четвер 16 серпень в...,21
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...,"[вночі, 16, серпень, пережити, найсильніший, г...",0.000000,вночі 16 серпень пережити найсильніший гроза з...,57
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...,"[уночі, вкотре, накрити, негода, найсильніший,...",0.000000,уночі вкотре накрити негода найсильніший дощ п...,0
...,...,...,...,...,...,...,...
1117,http://k.img.com.ua/rss/ua/3194862,Корреспондент: Діамантові руки. Історія успіху...,Київський офіс Класичного ювелірного дому Лобо...,"[київський, офіс, класичний, ювелірний, дома, ...",0.042616,київський офіс класичний ювелірний дома лоборт...,0
1118,http://k.img.com.ua/rss/ua/3194633,Корреспондент: Роздача слонів. Янукович щедро ...,20 років тому орден За заслуги – тоді він нази...,"[20, том, орден, заслуга, називатися, почесний...",0.124682,20 том орден заслуга називатися почесний знак ...,0
1119,http://k.img.com.ua/rss/ua/3194587,Корреспондент: Рівняння з трьома відомими. Укр...,10 жовтня політичні важковаговики з табору опо...,"[10, жовтень, політичний, важковаговик, табір,...",0.022042,10 жовтень політичний важковаговик табір опози...,0
1120,http://k.img.com.ua/rss/ua/3194570,Корреспондент: Точка зору. Мета обкрадає кошти...,"Добре там, де нас немає. В Ізраїлі ми є, але т...","[ізраїль, країна, займати, місце, світ, надій,...",0.018657,ізраїль країна займати місце світ надій молоко...,0


In [20]:
ukr_text.query('topic == 7')

,Id,Title,Body,words,tone,clean_text,topic
9,http://k.img.com.ua/rss/ua/3999418,У Києві посилять заходи безпеки 10 і 12 серпня,У Києві у зв'язку з проведенням футбольних мат...,"[звязка, проведення, футбольний, матч, 10, 12,...",0.000000,звязка проведення футбольний матч 10 12 серпен...,7
62,http://k.img.com.ua/rss/ua/4170402,Шахтар - Аталанта 0:3. Онлайн матчу Ліги Чемпі...,Українці проведуть домашню гру на ОСК Металі...,"[українка, провести, домашній, гру, оск, метал...",0.003763,українка провести домашній гру оск металіст ха...,7
63,http://k.img.com.ua/rss/ua/4165628,Олександрія - Вольфсбург 0:1. Онлайн матчу ЛЄ,"У четвер, 28-го листопада 2019 року, відбудеть...","[четвер, 28га, листопад, 2019, рок, відбутися,...",-0.001712,четвер 28га листопад 2019 рок відбутися поєдин...,7
71,http://k.img.com.ua/rss/ua/4161290,Футболіст збірної Бельгії десять хвилин грав в...,У відбірковому матчі чемпіонату Європи з футбо...,"[відбірковий, матч, чемпіонат, європа, футбол,...",-0.024194,відбірковий матч чемпіонат європа футбол 2020 ...,7
72,http://k.img.com.ua/rss/ua/4160247,Україна - Естонія 1:0. Онлайн-трансляція матчу,"Сьогодні, 14 листопада, на Кorespondent.net - ...","[14, листопад, кorespondentnet, онлайнтрансляц...",-0.008674,14 листопад кorespondentnet онлайнтрансляція м...,7
74,http://k.img.com.ua/rss/ua/4158548,Шахтар - Динамо 1-0. Онлайн матчу Прем'єр ліги,6:30 Попов підключився до атаки і заліз в офса...,"[630, попов, підключитися, атака, залізти, офс...",0.000000,630 попов підключитися атака залізти офсайд 62...,7
76,http://k.img.com.ua/rss/ua/4157768,Олександрія - Сент-Етьєн 2:2. Онлайн матчу ЛЄ,Українці проведуть домашній поєдинок на стадіо...,"[українка, провести, домашній, поєдинок, стаді...",0.001289,українка провести домашній поєдинок стадіон ар...,7
77,http://k.img.com.ua/rss/ua/4157749,Копенгаген - Динамо 1:1. Онлайн матчу Ліги Європи,Українці проведуть матч-відповідь на арені Пар...,"[українка, провести, матчвідповідь, арена, пар...",-0.010903,українка провести матчвідповідь арена паркен с...,7
82,http://k.img.com.ua/rss/ua/4170402,Шахтар - Аталанта 0:3. Онлайн матчу Ліги Чемпі...,Українці проведуть домашню гру на ОСК Металі...,"[українка, провести, домашній, гру, оск, метал...",0.003763,українка провести домашній гру оск металіст ха...,7
83,http://k.img.com.ua/rss/ua/4165628,Олександрія - Вольфсбург 0:1. Онлайн матчу ЛЄ,"У четвер, 28-го листопада 2019 року, відбудеть...","[четвер, 28га, листопад, 2019, рок, відбутися,...",-0.001712,четвер 28га листопад 2019 рок відбутися поєдин...,7


In [22]:
ukr_text.query('topic == 5')

,Id,Title,Body,words,tone,clean_text,topic
833,http://k.img.com.ua/rss/ua/4171554,ЦВК призначила довибори до Верховної Ради,У зв'язку з призначенням нардепа від Слуги нар...,"[звязка, призначення, нардеп, слуга, нарід, ол...",0.000000,звязка призначення нардеп слуга нарід олексій ...,5
834,http://k.img.com.ua/rss/ua/4157103,Вибори в Раду: ЦВК оголосила останній результат,Центрвиборчком визнав обраним депутатом Верхов...,"[центрвиборчком, визнати, обраний, депутат, ве...",0.029762,центрвиборчком визнати обраний депутат верховн...,5
835,http://k.img.com.ua/rss/ua/4156765,Вибори президента: члени однієї ОВК отримали у...,Мар'їнський районний суд Донецької області при...,"[марїнський, районний, суд, донецький, область...",-0.068376,марїнський районний суд донецький область приз...,5
836,http://k.img.com.ua/rss/ua/4154741,Вибори в Раду: ЦВК завершила перерахунок голос...,Центральна виборча комісія (ЦВК) завершила пов...,"[центральний, виборчий, комісія, цвк, завершит...",0.047619,центральний виборчий комісія цвк завершити пов...,5
837,http://k.img.com.ua/rss/ua/4146794,ЦВК відшкодувала парламентським партіям витрат...,Центральна виборча комісія (ЦВК) відшкодувала ...,"[центральний, виборчий, комісія, цвк, відшкоду...",0.028369,центральний виборчий комісія цвк відшкодувати ...,5
838,http://k.img.com.ua/rss/ua/4133803,ЦВК завершила реєстрацію нардепів,"Центральна виборча комісія (ЦВК) у середу, 28 ...","[центральний, виборчий, комісія, цвк, середа, ...",0.000000,центральний виборчий комісія цвк середа 28 сер...,5
839,http://k.img.com.ua/rss/ua/4132428,ЦВК залишилося зареєструвати менше 30 нардепів,Центральна виборча комісія (ЦВК) на засіданні ...,"[центральний, виборчий, комісія, цвк, засіданн...",0.000000,центральний виборчий комісія цвк засідання пят...,5
840,http://k.img.com.ua/rss/ua/4131732,ЦВК зареєструвала три чверті нової Ради,Центральна виборча комісія вже зареєструвала 3...,"[центральний, виборчий, комісія, зареєструвати...",0.000000,центральний виборчий комісія зареєструвати 321...,5
841,http://k.img.com.ua/rss/ua/4130907,ЦВК зареєструвала більше половини нардепів,"Центральна виборча комісія (ЦВК) у понеділок, ...","[центральний, виборчий, комісія, цвк, понеділо...",0.000000,центральний виборчий комісія цвк понеділок 19 ...,5
842,http://k.img.com.ua/rss/ua/4130173,Вибори в Прилуках: ЦВК розпустила комісію,"Центрвиборчком у п'ятницю, 16 серпня, прийняв ...","[центрвиборчком, пятниця, 16, серпень, прийнят...",0.000000,центрвиборчком пятниця 16 серпень прийняти ріш...,5
